<a href="https://colab.research.google.com/github/monacofj/moeabench/blob/add-test/misc/benchmark_gallery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoeaBench: Scientific Audit Confrontation
This notebook provides a systematic visual audit comparing the **Legacy2_optimal** against the **Analytical Ground Truth (v0.7.5)**.

It demonstrates the jump in mathematical precision and identifies the geometric displacement of legacy truths.


In [ ]:
# 1. Environment Setup & Root Discovery
import os
import sys
import pandas as pd
import numpy as np

def discover_root():
    # Check Colab
    if 'google.colab' in str(get_ipython()):
        if not os.path.exists('moeabench'):
            !git clone -b add-test https://github.com/monacofj/moeabench.git
        %cd moeabench
        !git pull origin add-test
        !pip install -e .
        return os.getcwd()
    
    # Local: search for MoeaBench package anchor
    curr = os.getcwd()
    for _ in range(3):
        if os.path.exists(os.path.join(curr, "MoeaBench")):
            return curr
        curr = os.path.dirname(curr)
    return os.getcwd()

PROJECT_ROOT = discover_root()
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import MoeaBench as mb
from MoeaBench.core import SmartArray
print(f"Project Root: {PROJECT_ROOT}")
print(f"MoeaBench version: {mb.system.version()}")


## Visual Auditor Helper
Logic to resolve absolute paths for artifacts and overlay data series (v0.7.5 vs Legacy2_optimal).

In [ ]:
def audit_confrontation(mop_name, M=3):
    print(f"--- Analyzing {mop_name} (M={M}) ---")
    
    # 1. Setup Problem & Manager
    exp = mb.experiment()
    try:
        mop = getattr(mb.mops, mop_name)(M=M)
    except:
        # Handle DPF which requires D
        mop = getattr(mb.mops, mop_name)(M=M, D=M-1)
    exp.mop = mop
    
    # 2. Load New Truth (v0.7.5)
    try:
        f_gt_pop = exp.optimal(n_points=1000)
        F_gt = f_gt_pop.objectives
        print(f"[OK] Analytical Truth: {len(F_gt)} points sampled.")
    except Exception as e:
        # Fallback (e.g. DTLZ8)
        gt_file = os.path.join(PROJECT_ROOT, "tests/ground_truth", f"{mop_name}_{M}_optimal.csv")
        if os.path.exists(gt_file):
            F_gt = pd.read_csv(gt_file, header=None).values
            print(f"[OK] Loaded Frozen Truth: {len(F_gt)} points from CSV.")
        else:
            print(f"[CRITICAL] Truth data missing for {mop_name}. Failed optimal() and missing {gt_file}.")
            return
            
    gt_series = SmartArray(F_gt, name="v0.7.5 Ground Truth")
    
    # 3. Resolve Legacy2_optimal (legacy2_optimal)
    legacy2_dir = os.path.join(PROJECT_ROOT, "tests/legacy2_optimal")
    legacy2_files = [f for f in os.listdir(legacy2_dir) if f.startswith(f"legacy_{mop_name}_M_{M}") and f.endswith(".csv")]
    
    overlays = [gt_series]
    
    if legacy2_files:
        leg_v2_path = os.path.join(legacy2_dir, legacy2_files[0])
        F_leg2 = pd.read_csv(leg_v2_path, header=None).values
        overlays.append(SmartArray(F_leg2, name="Legacy2_optimal (v2)"))
        print(f"[OK] Loaded Legacy2_optimal: {len(F_leg2)} points found at {leg_v2_path}.")
    else:
        print(f"[WARNING] Legacy2_optimal file not found for {mop_name}. Plotting GT only.")
        
    # 4. OVERLAY
    mb.view.topo_shape(*overlays, title=f"{mop_name} (M={M}): Confronto v0.7.5 vs Legacy2_optimal")


## 1. DTLZ Family Audits

In [ ]:
audit_confrontation("DTLZ1")

In [ ]:
audit_confrontation("DTLZ2")

In [ ]:
audit_confrontation("DTLZ3")

In [ ]:
audit_confrontation("DTLZ4")

In [ ]:
audit_confrontation("DTLZ5")

In [ ]:
audit_confrontation("DTLZ6")

In [ ]:
audit_confrontation("DTLZ7")

In [ ]:
audit_confrontation("DTLZ8")

In [ ]:
audit_confrontation("DTLZ9")

## 2. DPF Family Audits

In [ ]:
audit_confrontation("DPF1")

In [ ]:
audit_confrontation("DPF2")

In [ ]:
audit_confrontation("DPF3")

In [ ]:
audit_confrontation("DPF4")

In [ ]:
audit_confrontation("DPF5")